## Requirement if needed


In [ ]:
# Mount cloud
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#! pip install gdown

In [ ]:
#! gdown https://drive.google.com/uc?id=<SHARE_URL>

In [ ]:
#! unzip FILE_PATH

In [ ]:
#! pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [ ]:
#import torch_xla.core.xla_model as xm

## Dataset

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # Interactive mode

In [ ]:
from torch.utils.data import Dataset, DataLoader
from skimage import io

class CarDataset(Dataset):
    def __init__(self, dataframe, data_dir, transform=None):
        self.dataframe = dataframe
        self.data_dir = data_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.data_dir,
                                self.dataframe.iloc[idx, 0] + '.jpg')
        data = io.imread(img_path)
        if data.ndim == 2:
            data = np.asarray([data] * 3)
            data = np.transpose(data, (1, 2, 0))
        target = self.dataframe.iloc[idx, 1]
        sample = {'data': data, 'target': target}

        if self.transform:
            sample['data'] = self.transform(sample['data'])

        return sample

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.449, 0.434, 0.433], [0.282, 0.281, 0.286])
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.449, 0.434, 0.433], [0.282, 0.281, 0.286])
    ]),
}

In [ ]:
CSV_FILE = './training_labels.csv'
TRAINING_DIR = './training_data/training_data/'
TESTING_DIR = './testing_data/testing_data/'

In [ ]:
import pandas as pd

df = pd.read_csv(CSV_FILE, dtype=str)
#CAR_LABELS = list(set(df['label'])) # list(set()) is device/seed dependant
CAR_LABELS = ['Mercedes-Benz S-Class Sedan 2012', 'smart fortwo Convertible 2012', 'Acura TL Sedan 2012', 'Chevrolet Silverado 1500 Classic Extended Cab 2007', 'GMC Terrain SUV 2012', 'Dodge Durango SUV 2007', 'Acura ZDX Hatchback 2012', 'Cadillac SRX SUV 2012', 'Dodge Journey SUV 2012', 'Tesla Model S Sedan 2012', 'Ferrari 458 Italia Coupe 2012', 'Chevrolet Corvette ZR1 2012', 'Volvo C30 Hatchback 2012', 'BMW M3 Coupe 2012', 'Audi S6 Sedan 2011', 'Lamborghini Reventon Coupe 2008', 'Dodge Dakota Club Cab 2007', 'Aston Martin V8 Vantage Coupe 2012', 'Chevrolet Silverado 1500 Hybrid Crew Cab 2012', 'Audi 100 Sedan 1994', 'Chevrolet Sonic Sedan 2012', 'Buick Verano Sedan 2012', 'Hyundai Veracruz SUV 2012', 'BMW Z4 Convertible 2012', 'Hyundai Santa Fe SUV 2012', 'BMW 1 Series Convertible 2012', 'Chrysler PT Cruiser Convertible 2008', 'Nissan NV Passenger Van 2012', 'BMW 1 Series Coupe 2012', 'Chevrolet Camaro Convertible 2012', 'Bentley Continental Flying Spur Sedan 2007', 'Bentley Mulsanne Sedan 2011', 'Ford F-150 Regular Cab 2012', 'Aston Martin Virage Convertible 2012', 'Ford E-Series Wagon Van 2012', 'Nissan Leaf Hatchback 2012', 'Audi S4 Sedan 2012', 'Spyker C8 Convertible 2009', 'GMC Savana Van 2012', 'Dodge Caliber Wagon 2007', 'Audi TT RS Coupe 2012', 'Acura TSX Sedan 2012', 'Hyundai Sonata Sedan 2012', 'Buick Regal GS 2012', 'Ford Edge SUV 2012', 'Volkswagen Golf Hatchback 1991', 'Honda Odyssey Minivan 2007', 'GMC Canyon Extended Cab 2012', 'Hyundai Accent Sedan 2012', 'Chevrolet Corvette Ron Fellows Edition Z06 2007', 'Plymouth Neon Coupe 1999', 'Lamborghini Gallardo LP 570-4 Superleggera 2012', 'Rolls-Royce Phantom Drophead Coupe Convertible 2012', 'Volvo XC90 SUV 2007', 'Audi S4 Sedan 2007', 'Chevrolet Express Cargo Van 2007', 'Dodge Magnum Wagon 2008', 'BMW 3 Series Wagon 2012', 'Rolls-Royce Ghost Sedan 2012', 'Audi TTS Coupe 2012', 'Honda Accord Coupe 2012', 'Suzuki SX4 Hatchback 2012', 'Chevrolet Express Van 2007', 'Chevrolet Monte Carlo Coupe 2007', 'Isuzu Ascender SUV 2008', 'Rolls-Royce Phantom Sedan 2012', 'Audi RS 4 Convertible 2008', 'Infiniti G Coupe IPL 2012', 'Volvo 240 Sedan 1993', 'Bentley Continental GT Coupe 2007', 'GMC Acadia SUV 2012', 'Chevrolet Tahoe Hybrid SUV 2012', 'Fisker Karma Sedan 2012', 'Ford Freestar Minivan 2007', 'Lamborghini Diablo Coupe 2001', 'Bentley Arnage Sedan 2009', 'Volkswagen Golf Hatchback 2012', 'Chevrolet Silverado 1500 Extended Cab 2012', 'Dodge Ram Pickup 3500 Quad Cab 2009', 'Ford F-150 Regular Cab 2007', 'Dodge Caliber Wagon 2012', 'Suzuki SX4 Sedan 2012', 'Ford Expedition EL SUV 2009', 'Chevrolet Corvette Convertible 2012', 'Jeep Wrangler SUV 2012', 'BMW X5 SUV 2007', 'Suzuki Kizashi Sedan 2012', 'Toyota Sequoia SUV 2012', 'BMW M6 Convertible 2010', 'Mazda Tribute SUV 2011', 'Hyundai Sonata Hybrid Sedan 2012', 'Bentley Continental GT Coupe 2012', 'Ram C/V Cargo Van Minivan 2012', 'AM General Hummer SUV 2000', 'Cadillac Escalade EXT Crew Cab 2007', 'Land Rover LR2 SUV 2012', 'Lamborghini Aventador Coupe 2012', 'Jeep Compass SUV 2012', 'Volkswagen Beetle Hatchback 2012', 'Ford Mustang Convertible 2007', 'Honda Accord Sedan 2012', 'Ferrari FF Coupe 2012', 'Jeep Patriot SUV 2012', 'Buick Rainier SUV 2007', 'Chrysler Sebring Convertible 2010', 'Chrysler Aspen SUV 2009', 'Toyota Camry Sedan 2012', 'Dodge Charger SRT-8 2009', 'Audi R8 Coupe 2012', 'Lincoln Town Car Sedan 2011', 'Ford Ranger SuperCab 2011', 'Suzuki Aerio Sedan 2007', 'Toyota 4Runner SUV 2012', 'Jaguar XK XKR 2012', 'Eagle Talon Hatchback 1998', 'Bentley Continental Supersports Conv. Convertible 2012', 'Nissan Juke Hatchback 2012', 'MINI Cooper Roadster Convertible 2012', 'Scion xD Hatchback 2012', 'Acura RL Sedan 2012', 'HUMMER H2 SUT Crew Cab 2009', 'Dodge Durango SUV 2012', 'Audi A5 Coupe 2012', 'Chevrolet Traverse SUV 2012', 'Porsche Panamera Sedan 2012', 'Dodge Charger Sedan 2012', 'BMW 3 Series Sedan 2012', 'BMW X6 SUV 2012', 'Hyundai Veloster Hatchback 2012', 'Dodge Caravan Minivan 1997', 'Mercedes-Benz SL-Class Coupe 2009', 'Hyundai Tucson SUV 2012', 'Mercedes-Benz C-Class Sedan 2012', 'Dodge Ram Pickup 3500 Crew Cab 2010', 'Chevrolet Silverado 1500 Regular Cab 2012', 'Bugatti Veyron 16.4 Convertible 2009', 'Chevrolet Malibu Hybrid Sedan 2010', 'Buick Enclave SUV 2012', 'Chrysler 300 SRT-8 2010', 'Chrysler Town and Country Minivan 2012', 'Dodge Challenger SRT8 2011', 'Chevrolet Cobalt SS 2010', 'Aston Martin V8 Vantage Convertible 2012', 'Infiniti QX56 SUV 2011', 'Ford F-450 Super Duty Crew Cab 2012', 'Hyundai Genesis Sedan 2012', 'Mercedes-Benz E-Class Sedan 2012', 'Mercedes-Benz 300-Class Convertible 1993', 'Honda Odyssey Minivan 2012', 'Geo Metro Convertible 1993', 'Audi TT Hatchback 2011', 'Chrysler Crossfire Convertible 2008', 'BMW ActiveHybrid 5 Sedan 2012', 'Hyundai Azera Sedan 2012', 'Audi S5 Coupe 2012', 'Chevrolet Avalanche Crew Cab 2012', 'Hyundai Elantra Touring Hatchback 2012', 'Dodge Sprinter Cargo Van 2009', 'Ford Fiesta Sedan 2012', 'BMW X3 SUV 2012', 'Chevrolet HHR SS 2010', 'BMW M5 Sedan 2010', 'Chevrolet Silverado 2500HD Regular Cab 2012', 'Chevrolet Malibu Sedan 2007', 'Acura Integra Type R 2001', 'FIAT 500 Convertible 2012', 'Toyota Corolla Sedan 2012', 'Audi S5 Convertible 2012', 'GMC Yukon Hybrid SUV 2012', 'Daewoo Nubira Wagon 2002', 'Chevrolet Impala Sedan 2007', 'Jeep Liberty SUV 2012', 'Spyker C8 Coupe 2009', 'Acura TL Type-S 2008', 'Mercedes-Benz Sprinter Van 2012', 'McLaren MP4-12C Coupe 2012', 'Jeep Grand Cherokee SUV 2012', 'BMW 6 Series Convertible 2007', 'Ford Focus Sedan 2007', 'Cadillac CTS-V Sedan 2012', 'HUMMER H3T Crew Cab 2010', 'Bugatti Veyron 16.4 Coupe 2009', 'Mitsubishi Lancer Sedan 2012', 'Audi 100 Wagon 1994', 'Nissan 240SX Coupe 1998', 'Maybach Landaulet Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi V8 Sedan 1994', 'Land Rover Range Rover SUV 2012', 'Ferrari California Convertible 2012', 'Ferrari 458 Italia Convertible 2012', 'Hyundai Elantra Sedan 2007', 'Chevrolet TrailBlazer SS 2009', 'FIAT 500 Abarth 2012', 'Ford GT Coupe 2006', 'Dodge Dakota Crew Cab 2010']
LABEL_IDX_DICT = {label: idx for idx, label in enumerate(CAR_LABELS)}
print(len(CAR_LABELS), len(LABEL_IDX_DICT))
df.head()

In [ ]:
def train_val_dataframe(dataframe, val_rate=0.2):
    num_sample = len(dataframe)
    msk = np.random.permutation(num_sample) < num_sample * val_rate
    val_df = dataframe[msk]
    train_df = dataframe[~msk]
    return train_df, val_df

df = pd.read_csv(CSV_FILE, dtype=str)
# Change class to corresponding indices
for idx, label in enumerate(df['label']):
    df.iloc[idx, 1] = LABEL_IDX_DICT[label]
df['label'] = df['label'].astype(int)

train_df, val_df = train_val_dataframe(df)
print(train_df.shape, val_df.shape)
print(train_df.dtypes, val_df.dtypes)

In [ ]:
FLAGS = {}
FLAGS['batch_size'] = 64
FLAGS['num_workers'] = 0
FLAGS['learning_rate'] = 0.02
FLAGS['momentum'] = 0.9
FLAGS['num_epochs'] = 30

In [ ]:
image_datasets = {
    'train': CarDataset(train_df, TRAINING_DIR,
                        data_transforms['train']),
    'val': CarDataset(val_df, TRAINING_DIR,
                      data_transforms['val'])
}
dataloaders = {
    'train': DataLoader(image_datasets['train'],
                        batch_size=FLAGS['batch_size'],
                        shuffle=True,
                        num_workers=FLAGS['num_workers'],
                        drop_last=True),
    'val': DataLoader(image_datasets['val'],
                        batch_size=FLAGS['batch_size'],
                        shuffle=False,
                        num_workers=FLAGS['num_workers'],
                        drop_last=True),
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.asarray([0.449, 0.434, 0.433])
    std = np.asarray([0.282, 0.281, 0.286])
    inp = inp * std + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

samples = next(iter(dataloaders['train']))
inputs, classes = samples['data'], samples['target']
out = torchvision.utils.make_grid(inputs)

imshow(out)

## Train model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25, start=0, scheduler=None):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(start + epoch + 1, start + num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for x, samples in enumerate(dataloaders[phase]):
                print('Batch #{:<4d}'.format(x + 1))
                inputs, labels = samples['data'], samples['target']
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if scheduler is not None and phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f} Duration: {} s'.format(
                phase, epoch_loss, epoch_acc, time.time() - since))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({
                    'name': 'efficientnet-b0',
                    'epoch': start + epoch + 1,
                    'model_state_dict': best_model_wts,
                    'optimizer_state_dict': optimizer.state_dict()
                }, './drive/My Drive/best_ckpt_' + str(start + epoch + 1) + 'epoch.pt')

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Load best model weights (need?)
    #model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, samples in enumerate(dataloaders['val']):
            inputs, labels = samples['data'], samples['target']
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(CAR_LABELS[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

### Finetune

In [ ]:
#! pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
model_ft = EfficientNet.from_pretrained('efficientnet-b0')
#model_ft = models.resnet50(pretrained=True)
# Whether to freeze layers
for param in model_ft.parameters():
    param.requires_grad = True

In [ ]:
num_ftrs = model_ft._fc.in_features
model_ft._fc = nn.Linear(num_ftrs, len(CAR_LABELS))

#num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, len(CAR_LABELS))

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters())

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=2, start=30)

In [ ]:
visualize_model(model_ft)

## Apply on test dataset

In [ ]:
def get_prediction(model, transform, data_dir):
    result = {}
    model.eval()
    with torch.no_grad():
        for x, name in enumerate(os.listdir(data_dir)):
            img_path = os.path.join(data_dir, name)
            data = io.imread(img_path)
            if data.ndim == 2:
                data = np.asarray([data] * 3)
                data = np.transpose(data, (1, 2, 0))
            input = transform(data).to(device)
            input = input.unsqueeze(0)
            output = model(input)
            _, pred = torch.max(output, 1)
            result[name[:6]] = CAR_LABELS[pred]
            print('{:<15}{}'.format(name[:6], CAR_LABELS[pred]))

    return result

In [ ]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(400), # Use higher resolution
    transforms.CenterCrop(368),
    transforms.ToTensor(),
    transforms.Normalize([0.449, 0.434, 0.433], [0.282, 0.281, 0.286])
])

#CKPT_PATH = './checkpoint.pt'
#checkpoint = torch.load(CKPT_PATH['model_state_dict'])
model_ft.load_state_dict()
prediction = get_prediction(model_ft, test_transform, TESTING_DIR)

In [ ]:
d = {'id': list(prediction.keys()), 'label': list(prediction.values())}
df = pd.DataFrame(data=d)
df.head()

In [ ]:
# Save prediction to csv file
PREDICT_PATH = './prediction.csv'
df.to_csv(PREDICT_PATH, index=False)